In [1]:
import pandas as pd
from itertools import product

def parse_table(tables, parents):
    parsed_tables = {}
    for key, vars in parents.items():
        if vars is not None:
            vars_list = list(vars)
            parents_num = len(vars)
            combos = product([True, False], repeat=parents_num)
            combos_list = list(combos)
            prob_list = list(tables[key])
            df_list = list()
            i = 4*parents_num #num of prob values in output of A
            j = 0
            while j != i:
                df_list.append(prob_list[j])
                j += 2

            df_key=pd.DataFrame(df_list, columns=[key])
            cpt = pd.DataFrame(combos_list, columns=vars_list)
            cpt[key] = df_key
            parsed_tables[key] = cpt

        else:
            combos_list = [True, False]
            cpt = pd.DataFrame(combos_list, columns=[key])
            prob_list = list(tables[key])
            df_key=pd.DataFrame(prob_list)
            cpt['withself'] = df_key
        parsed_tables[key] = cpt
    return parsed_tables

parents = {'B': None, 'E': None, 'A': ['B', 'E'], 'J': ['A'], 'M': ['A']}
tables = {'B': [0.001, 0.999], 'E': [0.002, 0.998], 'A': [0.95, 0.05, 0.94, 0.06, 0.29, 0.71, 0.001, 0.999], 'J': [0.9, 0.1, 0.05, 0.95], 'M': [0.7, 0.3, 0.01, 0.99]}

parsed_tables = parse_table(tables, parents)
for key, table in parsed_tables.items():
    print("Table for variable", key)
    print(table)

Table for variable B
       B  withself
0   True     0.001
1  False     0.999
Table for variable E
       E  withself
0   True     0.002
1  False     0.998
Table for variable A
       B      E      A
0   True   True  0.950
1   True  False  0.940
2  False   True  0.290
3  False  False  0.001
Table for variable J
       A     J
0   True  0.90
1  False  0.05
Table for variable M
       A     M
0   True  0.70
1  False  0.01


In [3]:
random_var = 'A'
df = parsed_tables[random_var]
df[(df['B']==True)&(df['E']==True)][random_var]

0    0.95
Name: A, dtype: float64

In [19]:
import numpy as np
x = np.random.choice([True, False, "Other"], p=[0.33,0.32,0.35])
x

'Other'

In [29]:
# GENERAL PARSER
import pandas as pd
from itertools import product

def parse_table(tables, parents, domain):
    parsed_tables = {}
    for key, vars in parents.items():
        if vars is not None:

            parents_list = list(vars) #list of parents of A
            domain_list = domain.keys() #list of total variables
            parent_val = [] #list of lists

            for par in domain_list:
                 if par in parents_list:
                      parent_val.append(domain.get(par)) #LIST OF LISTS
            
            combos = product(*parent_val)
            combos_list = list(combos) #possible combinations

            #prob_list = list(tables[key]) # values of A

            cpt = pd.DataFrame(combos_list, columns=parents_list) #parents kdf  
            i = 0
            l = len(domain[key])
            for d in domain[key]: #in A
                somelist = list()
                for j in range (i, len(tables[key]),l):
                    somelist.append(tables[key][j])
                cpt[d] = somelist    
                i +=1
            #cpt = pd.DataFrame(combos_list, columns=parents_list) #parents kdf  
            #cpt[key] = df_var #combined df
            parsed_tables[key] = cpt
            
        else:
            combos_list = domain[key]
            cpt = pd.DataFrame(combos_list, columns=[key])
            prob_list = list(tables[key])
            df_key=pd.DataFrame(prob_list)
            cpt['withself'] = df_key
        parsed_tables[key] = cpt
    return parsed_tables

#domain = {'B': ['true', 'false', 'other'], 'E': ['true', 'false'], 'A': ['true', 'false'], 'J': ['true', 'false'], 'M': ['true', 'false']}
#parents = {'B': None, 'E': None, 'A': ['B', 'E'], 'J': ['A'], 'M': ['A']}
#tables = {'B': [0.001, 0.999], 'E': [0.002, 0.998], 'A': [0.95, 0.05, 0.94, 0.06, 0.29, 0.71, 0.001, 0.999, 0.04, 0.96, 0.95,0.05], 'J': [0.9, 0.1, 0.05, 0.95], 'M': [0.7, 0.3, 0.01, 0.99]}

domain = {'B': ['true', 'false'], 'E': ['true', 'false'], 'A': ['true', 'false'], 'J': ['true', 'false'], 'M': ['true', 'false']}
parents = {'B': None, 'E': None, 'A': ['B', 'E'], 'J': ['A'], 'M': ['A']}
tables = {'B': [0.001, 0.999], 'E': [0.002, 0.998], 'A': [0.95, 0.05, 0.94, 0.06, 0.29, 0.71, 0.001, 0.999], 'J': [0.9, 0.1, 0.05, 0.95], 'M': [0.7, 0.3, 0.01, 0.99]}

parsed_tables = parse_table(tables, parents, domain)
for key, table in parsed_tables.items():
    print("Table for variable", key)
    print(table)

Table for variable B
       B  withself
0   true     0.001
1  false     0.999
Table for variable E
       E  withself
0   true     0.002
1  false     0.998
Table for variable A
       B      E   true  false
0   true   true  0.950  0.050
1   true  false  0.940  0.060
2  false   true  0.290  0.710
3  false  false  0.001  0.999
Table for variable J
       A  true  false
0   true  0.90   0.10
1  false  0.05   0.95
Table for variable M
       A  true  false
0   true  0.70   0.30
1  false  0.01   0.99


In [30]:
def pos_prob(random_var, value, parsed_tables, given_random_var=None, given_val = None):
    #print(given_random_var)
    if given_random_var is None:
        df=parsed_tables[random_var]
        filter = (df[random_var]==value)
        return float(df[filter]['withself'])
    
    elif given_random_var is not None:
        df = parsed_tables[random_var]
        filter = True
        for p in given_random_var:
            filter = filter & (df[p]==given_val[p])
        return float(df[filter][value])
    else:
        return 0 

In [33]:
pos_prob('A', 'false', parsed_tables, ['B','E'], {'B':'true', 'E':'true'})

C:\Users\HP\AppData\Local\Temp\ipykernel_20788\3130009491.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(df[filter][value])


0.05